In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=0355c7b46e0d21fec2b46f9a4924001d3e4ae11b1145404566b852998061ac13
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [8]:
pip install centroid-tracker

Note: you may need to restart the kernel to use updated packages.


In [17]:
import time
import cv2
import datetime
import imutils
import numpy as np







def non_max_suppression_fast(boxes, overlapThresh):
    try:
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                                                   np.where(overlap > overlapThresh)[0])))

        return boxes[pick].astype("int")
    except Exception as e:
        print("Exception occurred in non_max_suppression : {}".format(e))


def main():
    cap = cv2.VideoCapture("videos/test_video.mp4")

    object_id_list = []
    dtime = dict()
    dwell_time = dict()
    
    while True:
        ret, frame = cap.read()
        frame = imutils.resize(frame, width=600)
        
        (H, W) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)

        detector.setInput(blob)
        person_detections = detector.forward()
        rects = []
        
        for i in np.arange(0, person_detections.shape[2]):
            confidence = person_detections[0, 0, i, 2]
            if confidence > 0.5:
                idx = int(person_detections[0, 0, i, 1])

                if CLASSES[idx] != "person":
                    continue

                person_box = person_detections[0, 0, i, 3:7] * np.array([W, H, W, H])
                (startX, startY, endX, endY) = person_box.astype("int")
                rects.append(person_box)
    
        boundingboxes = np.array(rects)
        boundingboxes = boundingboxes.astype(int)
        rects = non_max_suppression_fast(boundingboxes, 0.3)

        objects = tracker.update(rects)
        for (objectId, bbox) in objects.items():
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)

            if objectId not in object_id_list:
                object_id_list.append(objectId)
                dtime[objectId] = datetime.datetime.now()
                dwell_time[objectId] = 0
            else:
                curr_time = datetime.datetime.now()
                prev_time = dtime[objectId]
                time_diff = curr_time - prev_time
                sec = time_diff.total_seconds()
                dwell_time[objectId] += sec
                dtime[objectId] = datetime.datetime.now()

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = "{}".format(int(dwell_time[objectId]))
            cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
          

            
        
        cv2.imshow("Application", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    cv2.destroyAllWindows()


main()

AttributeError: 'NoneType' object has no attribute 'shape'

In [18]:
import cv2
import dlib
from scipy.spatial import distance as dist

def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal eye landmark
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    return ear

# initialize the dlib facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# define the threshold for drowsiness detection
EAR_THRESHOLD = 0.25

# initialize the video stream and wait for the camera to warm up
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cv2.waitKey(1000)

while True:
    # read the current frame from the video stream
    ret, frame = cap.read()

    # detect faces in the grayscale image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 0)

    # loop over each face
    for face in faces:
        # detect the facial landmarks for the face region
        shape = predictor(gray, face)
        shape = shape_to_np(shape)

        # extract the left and right eye coordinates
        left_eye = shape[36:42]
        right_eye = shape[42:48]

        # compute the eye aspect ratio for each eye
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)

        # compute the average eye aspect ratio
        ear = (left_ear + right_ear) / 2.0

        # check if the eye aspect ratio is below the threshold
        if ear < EAR_THRESHOLD:
            # indicate that the person is drowsy
            cv2.putText(frame, "DROWSY", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # indicate that the person is awake
            cv2.putText(frame, "AWAKE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# release the video stream and close all windows
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'dlib'

In [15]:
pip install CMake

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install dlib

  Using cached dlib-19.24.0.tar.gz (3.2 MB)
  Running setup.py clean for dlib
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\HP\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\HP\\AppData\\Local\\Temp\\pip-install-orsn2u93\\dlib_83327228fc4c41878b01d25daa1c2584\\setup.py'"'"'; __file__='"'"'C:\\Users\\HP\\AppData\\Local\\Temp\\pip-install-orsn2u93\\dlib_83327228fc4c41878b01d25daa1c2584\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\HP\AppData\Local\Temp\pip-wheel-wgplub5s'
       cwd: C:\Users\HP\AppData\Local\Temp\pip-install-orsn2u93\dlib_83327228fc4c41878b01d25daa1c2584\
  Complete output (58 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\python\dlib\__init__.py' not found (or not a regular f